<a href="https://colab.research.google.com/github/Gabriel-BP/Motor-de-Redes-Neuronales/blob/main/Motor_de_Redes_Neuronales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importamos las librerías necesarias

In [2]:
import numpy as np

Definimos funciones de activación y sus derivadas

In [1]:
# Función Sigmoide
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivada de Sigmoide
def sigmoid_derivative(x):
    return x * (1 - x)

# ReLU
def relu(x):
    return np.maximum(0, x)

# Derivada de ReLU
def relu_derivative(x):
    return np.where(x > 0, 1, 0)

Defininos una clase para cada capa de la red

In [3]:
class Layer:
    def __init__(self, input_size, output_size, activation_function, activation_derivative):
        # Inicializa los pesos y sesgos aleatoriamente
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.biases = np.random.rand(1, output_size) - 0.5
        self.activation_function = activation_function
        self.activation_derivative = activation_derivative

    # Propagación hacia adelante
    def forward(self, inputs):
        self.inputs = inputs
        self.z = np.dot(inputs, self.weights) + self.biases
        self.a = self.activation_function(self.z)
        return self.a

    # Propagación hacia atrás (backpropagation)
    def backward(self, output_error, learning_rate):
        # Calcula el gradiente respecto a la salida (error * derivada de activación)
        d_activation = self.activation_derivative(self.a)
        self.delta = output_error * d_activation
        input_error = np.dot(self.delta, self.weights.T)

        # Gradientes para pesos y sesgos
        weights_gradient = np.dot(self.inputs.T, self.delta)
        self.weights -= learning_rate * weights_gradient
        self.biases -= learning_rate * np.sum(self.delta, axis=0, keepdims=True)
        return input_error


Definimos la clase para el motor de la red neuronal

In [4]:
class NeuralNetwork:
    def __init__(self):
        self.layers = []

    # Agrega una capa a la red
    def add_layer(self, layer):
        self.layers.append(layer)

    # Propagación hacia adelante en toda la red
    def forward(self, X):
        output = X
        for layer in self.layers:
            output = layer.forward(output)
        return output

    # Entrenamiento con retropropagación
    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            # Propagación hacia adelante
            output = self.forward(X)

            # Calcula el error
            error = y - output
            for layer in reversed(self.layers):
                error = layer.backward(error, learning_rate)

            # Imprime el error cada 100 épocas
            if epoch % 100 == 0:
                print(f"Error en epoch {epoch}: {np.mean(np.abs(error))}")


Entrenamos el motor de redes neuronales

In [5]:
# Datos de entrada (ejemplo)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])  # XOR

# Configuración de la red
nn = NeuralNetwork()
nn.add_layer(Layer(input_size=2, output_size=3, activation_function=sigmoid, activation_derivative=sigmoid_derivative))
nn.add_layer(Layer(input_size=3, output_size=1, activation_function=sigmoid, activation_derivative=sigmoid_derivative))

# Entrenamiento
nn.train(X, y, epochs=1000, learning_rate=0.1)


Error en epoch 0: 0.0015614918652436865
Error en epoch 100: 0.0015178447289049017
Error en epoch 200: 0.0006098255471744007
Error en epoch 300: 0.0004702998221954755
Error en epoch 400: 0.0003821124745719436
Error en epoch 500: 0.0003159262231544662
Error en epoch 600: 0.00026971647610919995
Error en epoch 700: 0.0002360042382469057
Error en epoch 800: 0.00021033094320060792
Error en epoch 900: 0.00019009364305948924


Preprocesamiento de imágenes

In [ ]:
from PIL import Image
import numpy as np
import os

# Ruta a tus imágenes
cat_folder = "ruta/a/imagenes/gatos"
dog_folder = "ruta/a/imagenes/perros"

# Tamaño al que reducir las imágenes (por ejemplo, 64x64)
image_size = (64, 64)

def load_images(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = Image.open(img_path).convert("L")  # Escala de grises
        img = img.resize(image_size)
        img_array = np.array(img).flatten() / 255.0  # Normaliza los valores a [0, 1]
        images.append(img_array)
        labels.append(label)
    return np.array(images), np.array(labels)

# Cargar las imágenes de perros y gatos
cat_images, cat_labels = load_images(cat_folder, label=0)  # 0 para gatos
dog_images, dog_labels = load_images(dog_folder, label=1)  # 1 para perros

# Combina y mezcla los datos
X = np.vstack((cat_images, dog_images))
y = np.hstack((cat_labels, dog_labels))


Configuración del motor de la red neuronal

In [ ]:
# Inicializar la red
nn = NeuralNetwork()
nn.add_layer(Layer(input_size=4096, output_size=128, activation_function=sigmoid, activation_derivative=sigmoid_derivative))
nn.add_layer(Layer(input_size=128, output_size=64, activation_function=sigmoid, activation_derivative=sigmoid_derivative))
nn.add_layer(Layer(input_size=64, output_size=1, activation_function=sigmoid, activation_derivative=sigmoid_derivative))  # 1 salida para clasificación binaria


Entrenamiento del modelo

In [ ]:
# Entrenamiento
nn.train(X, y.reshape(-1, 1), epochs=1000, learning_rate=0.1)

Prueba del modelo

In [ ]:
# Predicción en una imagen
def predict_image(model, image_path):
    img = Image.open(image_path).convert("L").resize(image_size)
    img_array = np.array(img).flatten() / 255.0
    prediction = model.forward(img_array.reshape(1, -1))
    return "Perro" if prediction > 0.5 else "Gato"

# Ejemplo de predicción
print(predict_image(nn, "ruta/a/imagen/nueva.jpg"))
